In [30]:
import pandas as pd
import numpy as np
import pandas as pd
import fasttext
from ast import literal_eval
from collections import Counter

In [31]:
# Load the data into pandas DataFrames
print("Loading datasets...")
reviews_df = pd.read_csv("./archive/dataset.csv")  # columns: app_id, app_name, review_text


Loading datasets...


In [ ]:
tags_df = pd.read_csv("output.csv")        # columns: APPID, tags

# Map the string representation of lists/dicts back to actual lists/dicts using literal_eval
tags_df['tags'] = [literal_eval(x) for x in tags_df['tags']]

# Enable this line if you want to ignore the count of tags
new_tags = []
for x in tags_df['tags']:    
    if isinstance(x, dict):
        keys_list = list(x.keys())  
        new_tags.append(keys_list) 
    else:
        # If no tags
        new_tags.append([])
tags_df['tags'] = new_tags

print(tags_df.head())

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')



ImportError: DLL load failed while importing torch_directml_native: The specified procedure could not be found.

In [ ]:
import numpy as np

BASE_THRESHOLD = 0.4
MAX_LEN = 1000
MIN_THRESHOLD = 0.25

def dynamic_threshold(review_text):
    # Ensure review_text is a string
    text = review_text if isinstance(review_text, str) else ""
    length = len(text)
    # interpolate threshold linearly between BASE_THRESHOLD and MIN_THRESHOLD
    return float(np.interp(length, [100, MAX_LEN], [BASE_THRESHOLD, MIN_THRESHOLD]))



In [ ]:
tags = [tag for tags in tags_df['tags'] for tag in tags]
print(len(tags))
tags = list(set(tags))
print(len(tags))

tag_embeddings = model.encode(tags, convert_to_tensor=True)

similarity_adjusted_df = pd.DataFrame(columns=['app_id','app_name','review_text','review_score','review_votes'])

# for index, row in reviews_df.head(100000).iterrows():
for index, row in reviews_df.iterrows():
    if row['app_id'] != 10 and row['app_id'] != 1002:
        break
    review_text = row['review_text'] if isinstance(row['review_text'], str) else ""
    if review_text.strip(): 
        print("-> " + review_text)

    review_embedding = model.encode(review_text, convert_to_tensor=True)
    cosine_scores = util.cos_sim(review_embedding, tag_embeddings)[0]

    max_sim = float(cosine_scores.max())
    threshold = dynamic_threshold(review_text)

    # matched_indices = [i for i, score in enumerate(cosine_scores) if float(score) >= threshold]
    # matched_tags = [tags[i] for i in matched_indices]

    if max_sim >= threshold:
        # print("Included, matched tags:", matched_tags, "Max sim:", max_sim, ">=", threshold)
        print("Included, max similarity:", max_sim)
        similarity_adjusted_df = pd.concat(
            [similarity_adjusted_df, pd.DataFrame([row])],
            ignore_index=True
        )
    else:
        print("Excluded")
    
print(similarity_adjusted_df)
print(len(similarity_adjusted_df))
similarity_adjusted_df.to_csv("similarity_adjusted_reviews.csv", index=False)


110642
446
-> Ruined my life.
Included, max similarity: 0.4140688478946686
-> This will be more of a ''my experience with this game'' type of review, because saying things like ''great gameplay'' will not suit something I've experienced with Counter-Strike. Here you go:  I remember back in 2002 I was at a friend's house and he was playing a game. I didn't know the name of the game nor I had internet to find it. A few weeks passed by and another friend came over. He didn't have a computer, so he brought a disc with a game in it. He told me that it was one of the best games and from that very moment I knew that it is going to be the game I saw at the other friend's house. When I saw the Counter-Strike logo I was filled with gamegasm (?) and I was so happy. I was playing it hardcore. Made friends, clans, was involved in communities and even made two myself. Counter-Strike is my first game which I played competitively and it was a such an experience. Playing public servers with mods were v

In [ ]:
# Batches 
BATCH_SIZE = 1024       # adjust to your memory/GPU
TOP_K = 3              # number of tags per review
BASE_THRESHOLD = 0.45
MAX_LEN = 2000
MIN_THRESHOLD = 0.30
from tqdm import tqdm  # progress bar

In [ ]:
# --- Dynamic threshold function ---
def dynamic_threshold(review_text):
    text = review_text if isinstance(review_text, str) else ""
    text = text.strip()
    length = len(text)
    return float(np.interp(length, [100, MAX_LEN], [BASE_THRESHOLD, MIN_THRESHOLD]))

# --- Load model ---
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Prepare tags ---
tags = [tag for tags in tags_df['tags'] for tag in tags]
tags = list(set(tags))
tag_embeddings = model.encode(tags, convert_to_tensor=True)

# --- Prepare output DataFrame ---
similarity_adjusted_df = pd.DataFrame(columns=['app_id','app_name','review_text','review_score','review_votes','predicted_tags'])

# --- Process reviews in batches with progress bar ---
for start_idx in tqdm(range(0, len(reviews_df), BATCH_SIZE), desc="Processing batches"):
    batch = reviews_df.iloc[start_idx:start_idx + BATCH_SIZE]
    batch_texts = [t if isinstance(t, str) else "" for t in batch['review_text'].tolist()]

    # Encode batch
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True)
    cosine_scores = util.cos_sim(batch_embeddings, tag_embeddings)  # shape: [batch_size, num_tags]

    # Loop over batch to filter by dynamic threshold and assign top-k tags
    for i, row in batch.iterrows():
        scores = cosine_scores[i - start_idx]
        max_sim = float(scores.max())
        threshold = dynamic_threshold(row['review_text'])

        if max_sim >= threshold:
            # Get top-k tags with probabilities
            top_indices = scores.topk(TOP_K).indices
            predicted_tags = [f"{tags[idx]}:{scores[idx]:.2f}" for idx in top_indices]

            # Add review to output DataFrame
            row_copy = row.copy()
            row_copy['predicted_tags'] = predicted_tags
            similarity_adjusted_df = pd.concat([similarity_adjusted_df, pd.DataFrame([row_copy])], ignore_index=True)

# --- Save to CSV ---
similarity_adjusted_df.to_csv("similarity_adjusted_reviews.csv", index=False)
print("Saved", len(similarity_adjusted_df), "reviews to CSV")

Processing batches:  39%|███▉      | 382/977 [26:26<41:10,  4.15s/it]  


KeyboardInterrupt: 